In [2]:
import import_ipynb
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pytesseract
import os
import math
from process_img import process_img

importing Jupyter notebook from process_img.ipynb


In [3]:
index = 1
while True:
    if not process_img(index):
        break
    index += 1

Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


In [11]:
dir_path = './cropped'
sources = []
for (root, directories, files) in os.walk(dir_path):
    for file in files:
        if '.jpg' in file:
            file_path = os.path.join(root, file)
            sources.append(file_path)
            
src = [[] for _ in range(100)]
index = 0
temp = sources[0]
cnt = 0
print(sources)


while cnt < len(sources):
    if temp[10:-6] == sources[cnt][10:-6]:
        src[index].append(sources[cnt])
        temp = sources[cnt]
        cnt += 1
    else:
        temp = sources[cnt]
        index += 1
src = list(filter(None, src))

for i in src:
    print(i)


['./cropped\\file_10_cropped_0.jpg', './cropped\\file_11_cropped_0.jpg', './cropped\\file_11_cropped_1.jpg', './cropped\\file_1_cropped_0.jpg', './cropped\\file_2_cropped_0.jpg', './cropped\\file_3_cropped_0.jpg', './cropped\\file_3_cropped_1.jpg', './cropped\\file_3_cropped_2.jpg', './cropped\\file_4_cropped_0.jpg', './cropped\\file_4_cropped_1.jpg', './cropped\\file_4_cropped_2.jpg', './cropped\\file_5_cropped_0.jpg', './cropped\\file_5_cropped_1.jpg', './cropped\\file_5_cropped_2.jpg', './cropped\\file_6_cropped_0.jpg', './cropped\\file_6_cropped_1.jpg', './cropped\\file_6_cropped_2.jpg', './cropped\\file_6_cropped_3.jpg', './cropped\\file_7_cropped_0.jpg', './cropped\\file_7_cropped_1.jpg', './cropped\\file_7_cropped_2.jpg', './cropped\\file_8_cropped_0.jpg', './cropped\\file_8_cropped_1.jpg', './cropped\\file_8_cropped_2.jpg', './cropped\\file_8_cropped_3.jpg', './cropped\\file_9_cropped_0.jpg']
['./cropped\\file_10_cropped_0.jpg']
['./cropped\\file_11_cropped_0.jpg', './cropped\\

In [19]:
def make_crop(link):
    img = cv2.imread(link)
    img = cv2.resize(img, (150, 50))
    H,W,channel = img.shape
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_blurred = cv2.GaussianBlur(img_gray, ksize = (5,5), sigmaX = 0)
    img_thresh = cv2.adaptiveThreshold(img_blurred, maxValue = 255.0,
                                     adaptiveMethod = cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                    thresholdType = cv2.THRESH_BINARY_INV,
                                       blockSize = 19, C = 1)
    contours, _ = cv2.findContours(
                    img_thresh,
                    mode = cv2.RETR_LIST,
                    method = cv2.CHAIN_APPROX_SIMPLE
                    )
        
    temp_result = np.zeros((H,W, channel), dtype = np.uint8)
    cv2.drawContours(temp_result, contours = contours, 
                    contourIdx = -1, color = (255,255,255))

    contours_dict = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(temp_result, pt1 = (x,y), pt2 = (x+w, y+h),
                             color = (255,255,255), thickness = 1)
        contours_dict.append({
            'contour' : contour,
            'x' : x,
            'y' : y,
            'w' : w,
            'h' : h,
            'cx' : x + (w/2),
            'cy' : y + (h/2)
        })
            
    MIN_AREA = 100
    MAX_AREA = 800
    MIN_WIDTH, MIN_HEIGHT = 10, 10
    MIN_RATIO, MAX_RATIO = 0.4, 1
    
    possible_contours = []

    cnt = 0

    for d in contours_dict:
        area = d['w'] * d['h']
        ratio = d['w'] / d['h']
        if (area > MIN_AREA and d['w'] > MIN_WIDTH and d['h'] > MIN_HEIGHT 
            and MIN_RATIO < ratio < MAX_RATIO and area < MAX_AREA):
            d['idx'] = cnt
            cnt += 1
            possible_contours.append(d)
        
    temp_result = np.zeros((H, W, channel), dtype = np.uint8)
    for d in possible_contours:
        cv2.rectangle(temp_result, pt1 = (d['x'], d['y']), pt2 = (d['x']+d['w'], d['y']+d['h']),
                    color = (255,255,255), thickness = 1)
#         plt.figure(figsize=(6,5))
    plate_img = []
    if(len(possible_contours) < 2):
        return

    for i in range(len(possible_contours)):
        for j in range(i, len(possible_contours)):
            tmp = possible_contours[i]
            if possible_contours[i]['x'] < possible_contours[j]['x']:
                possible_contours[i] = possible_contours[j]
                possible_contours[j] = tmp

    for d in possible_contours:
        pw = d['w']
        ph = d['h']
        pcx = d['cx']
        pcy = d['cy']
        img_cropped = cv2.getRectSubPix(
            img,
            patchSize = (pw, ph),
            center = (pcx, pcy)
            )
        img_cropped = cv2.resize(img_cropped, (20,20))
        plate_img.insert(0, img_cropped)

#         for iii in range(len(plate_img)):
#             plt.subplot(1, len(plate_img), iii+1)
#             plt.imshow(plate_img[iii], cmap='gray')
#             plt.axis('off')

    for index in range(len(plate_img)):
        fname = './result/' + link[10:-4] + '_' + str(index) + '.jpg'
        cv2.imwrite(fname, plate_img[index])
        

In [21]:
for i in src:
    for j in i:
        make_crop(j)